In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
from utilities import *
import cv2
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim

In [26]:
groups = get_groups()

In [27]:
class ContrastComparer:
    def __init__(self):
        pass

    def rmse(self, x1, x2):
        x1 = np.array(x1)
        x2 = np.array(x2)
        return np.sqrt(np.mean((x1 - x2) ** 2))/np.sqrt(len(x1))
    
    def ame(self, x1, x2):
        x1 = np.array(x1)
        x2 = np.array(x2)
        return np.mean(np.abs(x1 - x2))

    def get_entropy(self, image, num_bins=64):
        histogram = np.histogram(image, bins=num_bins, range=(0, 1))[0]/image.size
    
        # Remove zero probabilities (log2(0) is undefined)
        histogram = histogram[histogram > 0]
        
        # Calculate entropy
        entropy = -np.sum(histogram * np.log2(histogram))
        
        max_entropy = np.log2(num_bins)

        return entropy/max_entropy

    def get_contrast_features(self, image):
        image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB).astype(np.float32)
        image /= 255.0
        l = image[:,:,0]
        features = [
            np.mean(l),
            # np.median(l),
            np.std(l),
            (np.max(l)-np.min(l))/(np.max(l)+np.min(l)),
            self.get_entropy(l)
        ]
        return features

    def compare_images(self, img1, img2, method='final'):     
        features1 = self.get_contrast_features(img1)
        features2 = self.get_contrast_features(img2)
        difference = self.rmse(features1, features2)
        return 1 - difference

In [28]:
comparer = ContrastComparer()
similarities = pd.DataFrame(columns=["final_image", "inspiration_image", "similarity"])
for index, row in tqdm(groups.iterrows(),total=groups.shape[0]):
    for finale_image_p in row["final_submissions"]:
        final_image = cv2.imread(finale_image_p)
        if final_image.shape[0] < 250: # Skip small images
            continue
        inspirations = row["ai_images"] + row["web_images"]
        for inspiration_p in inspirations:
            inspiration = cv2.imread(inspiration_p)
            similarity = comparer.compare_images(final_image, inspiration)
            similarities.loc[len(similarities)] = [finale_image_p, inspiration_p, similarity]

100%|██████████| 27/27 [02:52<00:00,  6.39s/it]


In [29]:
similarities.describe()

,similarity
count,4080.000000
mean,0.920850
std,0.058108
min,0.709849
25%,0.897769
50%,0.934353
75%,0.963589
max,0.999446


In [30]:
similarities.to_csv("contrast_similarity.csv", index=False)